In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_data_df = pd.read_csv("/content/combined_balanced_dataset_BERT_abigailapp.csv")
training_data_df['Content'] = training_data_df['Content'].fillna('').astype(str)    #ensure 'Content' is cleaned and valid
training_data_df = shuffle(training_data_df, random_state=42)       # shuffle


train_texts, val_texts, train_labels, val_labels = train_test_split(
    training_data_df['Content'].tolist(),
    training_data_df['Label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=training_data_df['Label'],
)

# subset of training data for faster prototyping
# train_texts = train_texts[:10000]
# train_labels = train_labels[:10000]

# DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=128)


# PyTorch Dataset wrapper
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir='./results_decay_args',
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    fp16=torch.cuda.is_available(),  # enable mixed precision if using GPU
    # report_to="none" 
    learning_rate=2e-5,  # much safer
)

steps_per_epoch = len(train_dataset) // training_args.per_device_train_batch_size
total_steps = steps_per_epoch * training_args.num_train_epochs
num_evals = total_steps // training_args.eval_steps


# Updated Metric Computation — per class
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision_not_malicious': precision[0],
        'recall_not_malicious': recall[0],
        'f1_not_malicious': f1[0],
        'precision_malicious': precision[1],
        'recall_malicious': recall[1],
        'f1_malicious': f1[1],
    }



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

results = trainer.evaluate()
print("Evaluation Results:", results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: acalde1997 (acalde1997-cal-poly-pomona) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision Not Malicious,Recall Not Malicious,F1 Not Malicious,Precision Malicious,Recall Malicious,F1 Malicious
500,0.308000,0.299615,0.876372,0.873335,0.880439,0.876872,0.879458,0.872304,0.875867
1000,0.255400,0.245637,0.899357,0.933827,0.859629,0.895193,0.869961,0.939084,0.903202
1500,0.188800,0.256330,0.906546,0.925376,0.884412,0.904430,0.889312,0.928680,0.908569
2000,0.183400,0.250476,0.902667,0.891628,0.916761,0.904020,0.914347,0.888574,0.901276


Evaluation Results: {'eval_loss': 0.2456369251012802, 'eval_accuracy': 0.8993567915247824, 'eval_precision_not_malicious': 0.9338265515824086, 'eval_recall_not_malicious': 0.8596292092319334, 'eval_f1_not_malicious': 0.8951930654058313, 'eval_precision_malicious': 0.8699614440939362, 'eval_recall_malicious': 0.9390843738176314, 'eval_f1_malicious': 0.9032023289665211, 'eval_runtime': 8.7786, 'eval_samples_per_second': 1204.285, 'eval_steps_per_second': 37.705, 'epoch': 1.5128593040847202}


In [ ]:
model.save_pretrained("Bert_Model")
tokenizer.save_pretrained("Bert_Model")
print("✅ Model and tokenizer saved to 'Bert_Model/'")

✅ Model and tokenizer saved to 'Bert_Model2/'


In [ ]:
# create a zip
!zip -r Bert_Model2.zip Bert_Model2

  adding: Bert_Model2/ (stored 0%)
  adding: Bert_Model2/vocab.txt (deflated 53%)
  adding: Bert_Model2/tokenizer_config.json (deflated 75%)
  adding: Bert_Model2/config.json (deflated 43%)
  adding: Bert_Model2/special_tokens_map.json (deflated 42%)
  adding: Bert_Model2/model.safetensors (deflated 8%)


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('/content/Bert_Model2')

# Load model
model = DistilBertForSequenceClassification.from_pretrained('/content/Bert_Model2')

# Example usage
inputs = tokenizer("You deserve to die.", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)

# Get prediction
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()
print("Predicted class:", predicted_class)


Predicted class: 1


In [ ]:
training_data_df['Label'].value_counts()

,count
Label,
1,21430
0,20430


In [ ]:

# 1. Define new texts
new_texts = [
    "Fuck this place.",
    "What a beautiful day, feeling grateful!",
    "I hate it here.",
    "You're a piece of shit",
    "Dumb ass bitch",
    "you’re such a dumbass 🤡 nobody wants you around 💩", #testing malicious tweets with emojis
    "go back to your country 🖕",
    "You're such a clown 🤡 lol",
    "🖕🤡🖕",
    "this is why you shouldnt have rights",
    "the wax lady said wiping front to back lmao i thought everybody did??"

]

# 2. Tokenize the new content
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# 3. Run the model in evaluation mode (no gradients)
model.eval()
with torch.no_grad():
    outputs = model(**new_encodings)
    predictions = torch.argmax(outputs.logits, dim=1)


# 4. Print results with readable labels
labels = ['Not Malicious', 'Malicious']
for text, pred in zip(new_texts, predictions):
    print(f"Text: {text}\nPrediction: {labels[pred]}\n")

Text: Fuck this place.
Prediction: Malicious

Text: What a beautiful day, feeling grateful!
Prediction: Not Malicious

Text: I hate it here.
Prediction: Not Malicious

Text: You're a piece of shit
Prediction: Malicious

Text: Dumb ass bitch
Prediction: Malicious

Text: you’re such a dumbass 🤡 nobody wants you around 💩
Prediction: Malicious

Text: go back to your country 🖕
Prediction: Malicious

Text: You're such a clown 🤡 lol
Prediction: Malicious

Text: 🖕🤡🖕
Prediction: Not Malicious

Text: this is why you shouldnt have rights
Prediction: Malicious

Text: the wax lady said wiping front to back lmao i thought everybody did??
Prediction: Not Malicious



In [ ]:
!pip list

Package                               Version
------------------------------------- ------------------
absl-py                               1.4.0
accelerate                            1.6.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.23
albumentations                        2.0.5
ale-py                                0.10.2
altair                                5.5.0
annotated-types                       0.7.0
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.1
arviz                                 0.21.0
astropy                               7.0.1
astropy-iers-data                     0.2025.4.21.0.37.6
astunparse                            1.6.3
atpublic                              5.1